In [81]:
import geopandas
import pandas as pd
import maup
import sys
sys.path.insert(0, '..')
from read_config import read_yaml
from shapely.ops import unary_union

In [86]:
config = read_yaml('../preprocessing_config.yaml')
main_path = '../../'
cb_path = main_path + config['PREPROCESSING']['DATA']['NEVADA']['PROCESSED']['ENACTED']['CENSUS_BLOCKS']
precincts_geo_path = main_path + config['PREPROCESSING']['DATA']['NEVADA']['UNPROCESSED']['GEOMETRY']['PRECINCTS']
districts_geo_path = main_path + config['PREPROCESSING']['DATA']['NEVADA']['UNPROCESSED']['GEOMETRY']['DISTRICTS']

cb = geopandas.read_file(cb_path)
districts_geo = geopandas.read_file(districts_geo_path)
districts_geo.to_crs(cb.crs, inplace=True)
precincts_geo=cb[['precinctId','geometry']].groupby(by='precinctId')['geometry'].apply(unary_union).reset_index(level =0)
precincts_info=cb.drop(columns = ['geometry']).groupby(by=['precinctId']).sum().reset_index().drop(columns=['districtId'])
precincts = precincts_info.merge(precincts_geo, on='precinctId')
precincts['districting'] = 'enacted'

In [87]:
assignment = maup.assign(precincts, districts_geo)
assert (assignment.isna().sum() == 0), "Error in census block to precinct assignment"
precincts['districtId'] = assignment + 1

/Users/kostadindev/.conda/envs/preprocessing/lib/python3.9/site-packages/maup/indexed_geometries.py:15: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  geometry.index = i
/Users/kostadindev/.conda/envs/preprocessing/lib/python3.9/site-packages/maup/indexed_geometries.py:27: UserWarning: GeoSeries.isna() previously returned True for both missing (None) and empty geometries. Now, it only returns True for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use 's.is_empty | s.isna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
  return intersections[-(intersections.is_empty | intersections.isna())]
/Users/kostadindev/.conda/envs/preprocessing/lib/python3.9/site-packages/ma

In [88]:
precincts_path = main_path + config['PREPROCESSING']['DATA']['NEVADA']['PROCESSED']['ENACTED']['PRECINCTS']
precincts.to_file(precincts_path, driver="GeoJSON")

In [85]:
precincts


,precinctId,population,white,black,asian,hispanic,democrat,republican,voters,geometry,districtId
0,3200101,1603,843,128,175,544,403.984075,373.100872,1073.287455,"POLYGON ((-118.78310 39.47480, -118.78384 39.4...",2
1,3200102,2260,1478,121,270,471,444.316366,394.127519,1159.876229,"POLYGON ((-118.78153 39.47491, -118.78153 39.4...",2
2,3200103,1258,742,46,112,386,392.393106,402.929179,1092.504382,"POLYGON ((-118.80029 39.47104, -118.80029 39.4...",2
3,3200104,3294,1582,369,319,1139,587.958964,502.825030,1513.353813,"MULTIPOLYGON (((-118.77243 39.46822, -118.7724...",2
4,3200105,4484,1938,403,458,1908,754.375340,616.825416,1905.027733,"POLYGON ((-118.78161 39.46370, -118.78161 39.4...",2
...,...,...,...,...,...,...,...,...,...,...,...
2000,32510403,2325,1242,188,377,581,361.362996,351.124211,983.803443,"POLYGON ((-119.75989 39.14651, -119.76005 39.1...",2
2001,32510405,2571,1408,269,134,810,352.903567,305.916195,913.904785,"POLYGON ((-119.76977 39.13349, -119.76970 39.1...",2
2002,32510407,6173,3788,576,525,1448,1394.977625,1212.044090,3614.276212,"POLYGON ((-119.77755 39.11395, -119.77756 39.1...",2
2003,32510409,2794,1285,330,300,989,534.775135,445.744626,1359.631727,"POLYGON ((-119.88475 39.11287, -119.88515 39.1...",2
